In [1]:
import pickle
import torch
import pandas as pd

import sys
sys.path.append("..")
import clustering

from transformers import BertTokenizer

/home/gsc685/.conda/envs/gabriella_cwr4lsc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
features = 'binder'

In [3]:
with open('../data/cwr4lsc/{}/usages_len128.clustering.2.dict'.format(features), 'rb') as f:
    clusters = pickle.load(f)

these are the clusters for virtual in coha

In [4]:
clusters['virtual']

KMeans(n_clusters=2, random_state=42)

mario has provided us with a function to sample the most prototypical usages from the cluster

for that we need to load the usages

In [6]:
with open('../data/cwr4lsc/{}/matrix_usages_16_len128.dict'.format(features), 'rb') as f:
    usages = pickle.load(f)

Now we can pull some of the sentences closest to the centroids of the clusters for each word

In [7]:
# now i want ot get the nearest neighbors of each cluster...the prototypes function doesnt work so here is my bversion of it. 

import numpy as np
from scipy.spatial.distance import cdist


word = 'virtual'
window = 5
U_w, contexts, positions, t_labels = usages[word]
cluster_id = 1
n = 10


print(clusters['virtual'].cluster_centers_.shape)

centers = clusters['virtual'].cluster_centers_.shape

print(clusters['virtual'].labels_.shape)

labels = clusters['virtual'].labels_.shape

print(usages['virtual'][0].shape)

nearest = np.argsort(cdist([clusters['virtual'].cluster_centers_[cluster_id]], usages['virtual'][0]), axis=1)[:, -n:]

print(nearest[0])


for i in nearest[0]:
    sent = contexts[i]
    print(sent)
    pos = positions[i]
    print(pos)

    #assert sent[pos] == word
    sent[pos] = '[[{}]]'.format(word)
    sent = sent[pos - window: pos + window + 1]
    sent = ' '.join(sent)

    print(sent)

(2, 62)
(1940,)
(1940, 62)
[ 814 1370 1535 1360 1820 1779 1085  593 1274 1330]
['from', 'a', 'central', '##ised', 'whitehall', 'perspective', ',', 'there', 'was', 'no', 'reason', 'why', 'associations', 'should', 'need', 'these', 'reserves', '.', 'the', 'role', 'of', 'the', 'territorial', 'army', 'was', 'determined', 'by', 'the', 'war', 'office', ',', 'and', 'grants', 'were', 'made', 'to', 'enable', 'the', 'associations', 'to', 'raise', 'and', 'administer', 'the', 'units', 'required', 'to', 'fu', '##lf', '##il', 'that', 'role', '.', 'the', 'accumulation', 'of', 'surplus', '##es', ',', 'which', 'gave', 'some', 'associations', 'virtual', 'financial', 'independence', ',', 'was', 'a', 'blow', 'to', 'the', 'war', 'office', "'", 's', 'assertion', 'of', 'pre', '##dom', '##ina', '##nce', 'as', 'well', 'as', 'an', 'offence', 'against', 'the', 'canons', 'of', 'treasury', 'control', 'and', 'financial', 'orthodoxy', '@', '@', '@', '@', '@', '@', '@', '@', '@', '@', ',', 'the', 'associations', 'foun

In [ ]:
def get_prototypes_1(word, clustering, usages, n=5, window=10):
    num_clusters = len(clustering.cluster_centers_)
    U_w, contexts, positions, t_labels = usages

    
    prototypes = []

    for cluster_id in range(0, num_clusters):
        #print("cluster_id", cluster_id)
        #print(clustering.cluster_centers_.shape)

        centers = clustering.cluster_centers_.shape

        #print(clustering.labels_.shape)

        labels = clustering.labels_.shape

        #print(usages[0].shape) # 0 index is the vectors

        nearest = np.argsort(cdist([clustering.cluster_centers_[cluster_id]], U_w), axis=1)[:, -n:]

        #print(nearest[0])
        
        this_cluster_prototypes = []
        for i in nearest[0]:
            sent = contexts[i]
            #print(sent)
            pos = positions[i]
            #print(pos)

            #assert sent[pos] == word
            sent[pos] = '[[{}]]'.format(word)
            sent = sent[pos - window: pos + window + 1]
            sent = ' '.join(sent)
            this_cluster_prototypes.append(sent)
        prototypes.append(this_cluster_prototypes)
        
    return(prototypes)

In [ ]:
get_prototypes_1('brick', clusters['brick'], usages['brick'], n=5, window=10)

[['by boo ##ing : a three - story red - [[brick]] general motors assembly plant sp ##routed workers at every window',
  'len ##ox avenue , a large u - shaped red [[brick]] apartment building seventeen stories high . the front garden was',
  'to be seen were the fl ##ori ##d red - [[brick]] catholic church and rectory at the end of main street',
  'standing on the porch of his two - story red [[brick]] house ready to give them an official welcome to this',
  'the fence , at the rear of the red - [[brick]] house , then at dora , and back at the'],
 ['with the ten ##eb ##rous significance of dreams - every [[brick]] and stone , the scrap ##s of news ##print tumbling',
  'drove through little towns , secure and complete in their [[brick]] and white simplicity . they passed the lake several times',
  'the most visible component of police presence , the imposing [[brick]] and stone structures , was no less threatening to residents',
  "##s of all - too - familiar sets line the [[brick]] and 

In [ ]:
get_prototypes_1('virtual', clusters['virtual'], usages['virtual'], n=5, window=10)

[['@ @ carlos romero bar ##cel ##o finished in a [[virtual]] dead heat with former governor rafael hernandez colon , who',
  'production in war - rack ##ed italy came to a [[virtual]] halt in 1942 , the year bra ##zzi turned twenty',
  '1980 , latin american income growth has slowed to a [[virtual]] halt - to less than 6 percent over twenty years',
  'of new york . the popular vote ended in a [[virtual]] tie , but cleveland took the electoral college , 219',
  '; reductions in the city university budget , and a [[virtual]] halt to any new @ @ @ @ @ @'],
 ['the classroom feel real , they needed to connect the [[virtual]] world to their real world through an ina ##ni ##mate',
  'like playing a large and complex video game . some [[virtual]] - reality software developers have been working on a product',
  'over the internet , and with it the potential for [[virtual]] reality to promote human interaction over wide distances . imagine',
  'his forehead . " the sophisticated sensory array of

In [ ]:
get_prototypes_1('disk', clusters['disk'], usages['disk'], n=5, window=10)

[["software . once installed on a user ' s hard [[disk]] drive , the software creates a large file ( 900",
  '. using the work ##ben ##ch , you can copy [[disk]] ##ettes , view files and load programs by pointing and',
  '- 22 . computer owners will not buy the special [[disk]] drives required to play cds on their desktop machines until',
  "photo ( color ) : monster lair ' s compact [[disk]] attachment makes its sound th ##ro ##b . < p",
  'and copy the . ps ##t file from the floppy [[disk]] over it , changing the name if necessary . of'],
 ['been torn back to make way for the huge gray [[disk]] that blocked its entrance . each day someone swept the',
  'they , themselves , were concerned , on a swaying [[disk]] of steel at the centre of an equatorial waste of',
  'the fr ##isi ##an islands , i saw a golden [[disk]] on the superstructure of the ship , as if the',
  "' clock when the moon rose , a pale yellow [[disk]] above the hills that rim ##med the valley of the",
  ', and rising s

In [ ]:
get_prototypes_1('parent', clusters['parent'], usages['parent'], n=5, window=10)

[['er ##udi ##te and highly paid legal staff of the [[parent]] company . cases were started simultaneously before the " push',
  'except for a few bridges ##tones shipped in from the [[parent]] company in japan . fires ##tone argues that its tire',
  'gives iran a seat on the supervisory board of the [[parent]] company as well . gradually , cautiously , major oil',
  'of subsidiaries of consolidated gas , but includes only the [[parent]] company , as a manufacturer and distributor of artificial gas',
  'plan afford ##s an opportunity to purchase stock of the [[parent]] company , standard oil of new jersey , a so'],
 ['of high - profile child - rear ##ing experts and [[parent]] educators advise against span ##king . the canadian pa ##ed',
  'make the situation control the behavior , rather than the [[parent]] controlling it , " she said . in other words',
  'rules , guidelines and boundaries . it means becoming the [[parent]] god intended when he blessed you with the gift of',
  'the ch

In [16]:
import importlib
importlib.reload(clustering)

<module 'clustering' from '/home/gsc685/features_in_context/notebooks/../clustering.py'>

In [17]:
U_w, contexts, positions, t_labels = usages['brick']
print(U_w[:5])
print(contexts[:5])
print(positions[:5])
print(t_labels[:5])

[[-0.00812746  0.04887854  0.07403399 ... -0.00152213  0.00082821
  -0.00655399]
 [ 0.02108257  0.06928941  0.02662214 ... -0.00482794  0.00472053
  -0.00557994]
 [ 0.02375785  0.03040661  0.05543758 ... -0.00719667 -0.00230765
  -0.00475294]
 [ 0.00580683 -0.01956674  0.03241314 ... -0.00418058 -0.0047539
  -0.00660921]
 [-0.01225548  0.02018052  0.00766065 ... -0.00272244  0.00297417
  -0.0048099 ]]
[['the', 'door', 'and', 'window', '@', '@', '@', '@', '@', '@', '@', '@', '@', '@', 'cas', '##ings', 'and', 'the', 'roof', 'are', 'stained', 'brown', '.', 'the', 'leaders', 'are', 'an', 'ox', '##idi', '##zed', 'copper', '.', 'the', 'shutter', '##s', 'and', 'the', 'box', 'tree', 'tub', '##s', 'are', 'a', 'dark', ',', 'vivid', 'green', ',', 'and', 'the', 'walks', 'and', 'terrace', 'are', 'laid', 'with', 'a', "'", 'trim', '##ly', 'edged', 'bright', 'red', 'brick', '.', 'the', 'outdoor', 'living', 'room', 'or', 'inc', '##los', '##ed', 'piazza', 'has', 'become', 'an', 'essential', 'in', 'our',

In [18]:
from collections import Counter
Counter(clusters['brick'].labels_)


Counter({2: 3833, 0: 3292, 1: 396})

In [ ]:
clustering.get_prototypes('brick', clusters['brick'], usages['brick'], n=5, window=10)

shape of cluster centers: (3, 3981)
{0: None, 1: None, 2: None}
{0: [], 1: [], 2: []}
number of usages:  7521
getting the distances between labels and cluster centers
[[ 0.28595546 -0.30922326 -0.29719776 ...  0.1476648   0.47021779
   0.505579  ]
 [-0.43143812 -0.66463988  0.26886116 ... -0.90330876 -0.26752028
   0.85099612]
 [-0.20102162  0.33424481  0.22747352 ... -0.03349915 -0.37621157
  -0.52213946]]
[[-1.01086269e-02 -7.30435634e-02 -1.70028330e-02 ... -1.88235209e-03
  -4.12591029e-04 -8.96971381e-04]
 [ 1.24214187e-02 -2.99085947e-04 -1.02361564e-02 ... -2.44219559e-03
   8.99620583e-04  1.37535246e-03]
 [-9.10892536e-03 -1.49765026e-03 -4.25341522e-02 ...  5.14178541e-04
   9.58790441e-05  5.05714455e-03]
 ...
 [-7.76622563e-03 -8.41825636e-04 -3.22426317e-02 ... -1.68196793e-03
  -7.87630541e-04  8.05641161e-04]
 [ 1.69681619e-02  3.23044108e-02  3.42822247e-02 ... -6.20273528e-03
  -2.93256973e-03 -6.60937523e-03]
 [-6.52541525e-03  6.16317176e-03 -9.00135491e-04 ... -6.09

ValueError: XA must be a 2-dimensional array.

In [ ]:
clustering.get_prototypes('virtual', clusters['virtual'], usages['virtual'], n=5, window=10)

In [ ]:
clustering.get_prototypes('disk', clusters['disk'], usages['disk'], n=5, window=10)

In [ ]:
get_prototypes('parent', clusters['parent'], usages['parent'], n=5, window=10)

In [ ]:
tz = BertTokenizer.from_pretrained('bert-base-uncased')

for word in clusters.keys():
    print(" ")
    print("prototypes for ", word)
    if word == 'leaf':
        continue
    prototypes = clustering.get_prototypes(word, clusters[word], usages[word], n=5, window=10)
    for i, prototype in enumerate(prototypes):
          print("prototype ", i)
          print(tz.convert_tokens_to_string(prototype))



 
prototypes for  leaf
 
prototypes for  sleep
prototype  0

prototype  1

prototype  2

prototype  3
to jim dandy mumbling and moaning in his [[sleep]] , then looks at jim crow . p . 42 and as norden watched he groaned again in his [[sleep]] , and shifted his head on the two pillows under haphazardly swipes at it in his [[sleep]] , wakes @ @ @ @ @ @ @ @ and his dog and little fox jones twitched in their [[sleep]] . the reverend birdsong accepted mrs . likewise ' the man snored lightly and turned in his [[sleep]] . max froze for a moment , then the figure
prototype  4

prototype  5

prototype  6

prototype  7

prototype  8

prototype  9

 
prototypes for  energy
prototype  0

prototype  1
cancellation was provided for in the contract between the atomic [[energy]] commission and the dixon - yates combination , known as the t . v . a . to the atomic [[energy]] commission . then there will be the hardy perennials was responsible for 18 percent . in april the international [[energy]] agenc

AssertionError: 

In [8]:
virtual_clusters = clusters['virtual']

In [9]:
virtual_clusters.cluster_centers_

array([[ 0.30642665,  0.22339732,  0.20428522, ..., -0.03316359,
        -0.04161427,  0.35041528],
       [-0.3265256 , -0.23805026, -0.21768457, ...,  0.03533884,
         0.04434381, -0.37339951]])

In [ ]:
buchanan = buchanan = torch.load('../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters')


ValueError: X has 3981 features, but PLSRegression is expecting 768 features as input.

In [ ]:
now we remap the data to have one row for each feature

In [16]:


feature_map = buchanan.feature_norms.feature_map
feature_labels = [str(feature_map.get_object(i)) for i in range(0, len(feature_map))]
print(feature_labels[:10])



['desert', 'give', 'leave', 'up', 'withdraw', 'belly', 'body', 'middle', 'muscle', 'organ']


In [17]:


# model dimensions dont have full 65 oops who knows what happened if our original downloaded data was bunk
len(feature_labels)



3981

In [37]:
targets = []
prototype_ids =[]
#nouns = []
#positions = []
features = []
values = []


predictions = []

for word in clusters.keys():
    centroids = clusters[word].cluster_centers_
    for cluster_id, cluster in enumerate(centroids):

        prediction = centroids[cluster_id]
        i=0
        for value in prediction:
            targets.append(word)
            prototype_ids.append(cluster_id)
            features.append(feature_labels[i])
            values.append(value)
            i+=1

        #prototypes = clustering.get_prototypes(word, clusters[word], usages[word], n=5, window=10)

    tidy_df = pd.DataFrame.from_records(
        {"target": targets,
        "prototype_id": prototype_ids, 
        #"prototype_sentences": prototypes, 
        "feature": features,
        "value": values,
        }
    )
    tidy_df.to_csv('../data/cwr4lsc/results/cluster_centroid_features.csv')

    

In [36]:
tidy_df.head(5)

,feature,prototype_id,target,value
0,desert,0,leaf,-1.186869
1,give,0,leaf,-1.121639
2,leave,0,leaf,-1.951784
3,up,0,leaf,0.149296
4,withdraw,0,leaf,-1.602879
